### multiple subdomain 결과 재정렬

In [ ]:
# import json
# domain = '경제'
# with open(f'evaluation/eval_data/multiple_subdomain_results/{domain}_subdomain_classified.json', 'r', encoding='utf-8') as f:
    # data = json.load(f)

#  new_data = []
# for d in data:
#     if d['qna_subdomain_reason'] in d['qna_reason']:
#         d['qna_reason'] = d['qna_reason'].replace(d['qna_subdomain_reason'], '').strip()
#         # print(d)
#     new_d = {
#         'file_id': d['file_id'],
#         'title': d['title'],
#         'chapter': d['chapter'],
#         'tag': d['tag'],
#         'domain': d['domain'],
#         'subdomain': d['subdomain'],
#         'domain_reason': d['domain_reason'],
#         'subdomain_reason': d['subdomain_reason'],
#         'question': d['question'],
#         'options': d['options'],
#         'answer': d['qna_answer'],
#         'explanation': d['explanation']
#     }
#     new_data.append(new_d)
    
# with open(f'evaluation/eval_data/subdomain_results_old/{domain}_subdomain_classified.json', 'w', encoding='utf-8') as f:
#     json.dump(new_data, f, ensure_ascii=False, indent=4)

## 모의고사 만들기 파이프라인

In [ ]:
# import os, json, random
# from datetime import datetime

# random.seed(42)

# # 과목 분류
# BBASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data'
# with open(os.path.join(BBASE_DIR, 'exam_hierarchy.json'), 'r', encoding='utf-8') as f:
#     subjects = json.load(f)

# # subdomain 분류 완료된 데이터 > 모의고사 대상 문제주머니 추출
# BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_with_subdomain'

# with open(os.path.join(BASE_DIR, 'classification_statistics.json'), 'r', encoding='utf-8') as f:
#     stats = json.load(f)

# # 로그 파일 설정
# log_output = []
# # timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# log_file = f'/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs/mock_exam_pipeline.log'

# for json_file in os.listdir(BASE_DIR):
#     if not json_file.endswith("_subdomain_classified.json"):
#         continue
    
#     with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     domain = json_file.replace("_subdomain_classified.json", "")
#     log_message = f"=====================================\n<<< {domain} 분야 모의고사용 문제 추출 >>>"
#     print(log_message)
#     log_output.append(log_message)
    
#     total_questions = len(data)
#     if total_questions == stats[domain]['total_questions']:
#         mock_data = []
#         need_more = {}
#         mock_total_questions = round(total_questions)
        
#         log_message1 = f"원본 문제 수: {total_questions}"
#         log_message2 = f"추출 대상 문제 수: {mock_total_questions}"
#         print(log_message1)
#         print(log_message2)
#         log_output.extend([log_message1, log_message2])

#         # subdomain 별로 모의고사 만들기
#         for subdomain in stats[domain]['subdomain_distribution'].keys():
#             subdomain_data = [d for d in data if d['subdomain'] == subdomain]
#             random.shuffle(subdomain_data)
#             try:
#                 mock_subdomain_data = random.sample(subdomain_data, mock_total_questions // len(stats[domain]['subdomain_distribution']))
#             except ValueError:
#                 mock_subdomain_data = subdomain_data
#                 need_more[subdomain] = mock_total_questions // len(stats[domain]['subdomain_distribution']) - len(mock_subdomain_data)
            
#             log_message = f"  - {subdomain}: {len(mock_subdomain_data)}"
#             print(log_message)
#             log_output.append(log_message)
#             mock_data.extend(mock_subdomain_data)

#         log_message1 = f" ====> 총 추출 문제: {len(mock_data)}"
#         print(log_message1)
#         log_output.append(log_message1)
        
#         if need_more:
#             log_message2 = f" ====> 모자란 문제: {need_more}"
#             print(log_message2)
#             log_output.append(log_message2)
#     else:
#         log_message = "갯수가 안맞아!"
#         print(log_message)
#         log_output.append(log_message)
    
#     # 문제주머니 저장
#     for domains in subjects.values():
#         if domain in domains:
#             subject = list(subjects.keys())[list(subjects.values()).index(domains)]
    
#     OUTPUT_DIR = f"{BBASE_DIR}/3_multiple_mock/{subject}"

#     os.makedirs(OUTPUT_DIR, exist_ok=True)
#     with open(f'{OUTPUT_DIR}/{domain}_mock_data.json', 'w', encoding='utf-8') as f:
#         json.dump(mock_data, f, ensure_ascii=False, indent=4)

# # 로그 파일 저장
# os.makedirs('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs', exist_ok=True)
# with open(log_file, 'a', encoding='utf-8') as f:
#     f.write('\n'.join(log_output))

# print(f"\n로그가 저장되었습니다: {log_file}")

### 수정한 도메인 원본에 반영하기?

In [ ]:
# import os, json

# MULTIPLE_EVAL_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_bronze_layer_subdomain/multiple_with_subdomain'
# EXTRACTED_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/workbook_data'

# for json_file in os.listdir(MULTIPLE_EVAL_DIR):
#     if not json_file.endswith("_subdomain_classified.json"):
#         continue
    
#     with open(os.path.join(MULTIPLE_EVAL_DIR, json_file), 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     for d in data:
#         # print(d)
#         file_path = os.popen(f"find {EXTRACTED_DIR} -type f -name '{d['file_id']}_extracted_qna.json'").read().strip()
#         with open(file_path, 'r', encoding='utf-8') as f:
#             extracted_data = json.load(f)


#         for extracted_d in extracted_data:
#             print(extracted_d)
#             break
#         # for extracted_d in extracted_data:
#         #     if extracted_d['id'] == d['file_id']:
#         #         print(extracted_d)
#         break
        
#     break
    
#     new_data = []
#     other_domain = {}
    

In [ ]:
extracted_d = extracted_data[0]
extracted_d

### 분류 조정하기

In [ ]:
# import os, json

# BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/multiple_with_subdomain'

# for json_file in os.listdir(BASE_DIR):
#     if not json_file.endswith("_subdomain_classified.json"):
#         continue
    
#     with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     domain = json_file.replace("_subdomain_classified.json", "")
    
#     new_data = []
#     other_domain = {}
#     for d in data:
#         if d['qna_domain'] != domain:
#             other_domain[d['qna_domain']] = d
#         else:
#             new_data.append(d)
    
#     with open(os.path.join(BASE_DIR, f'{domain}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
#         json.dump(new_data, f, ensure_ascii=False, indent=4)
        
#     for other_domain_key in other_domain.keys():
#         with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'r', encoding='utf-8') as f:
#             other_domain_data = json.load(f)
        
#         other_domain_data.append(other_domain[other_domain_key])
                
#         with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
#             json.dump(other_domain_data, f, ensure_ascii=False, indent=4)

### mock 데이터에서 모의고사 문제 추출

In [ ]:
import os, json, random
from datetime import datetime

BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data'
PROCESSED_DIR = os.path.join(BASE_DIR, '2_subdomain')
MOCK_DIR = os.path.join(BASE_DIR, '3_multiple_mock')
EXAM_DIR = os.path.join(BASE_DIR, '4_multiple_exam')

mock_exam_name = {
    1 : '1st',
    2 : '2nd',
    3 : '3rd'
}


with open(os.path.join(BASE_DIR, 'exam_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)

# 로그 파일 설정
log_output = []
# timestamp = datetime.now().strftime("%Y%m%d_%H%M")
log_file = f'/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs/mock_exam_extraction.log'

for subject in os.listdir(BASE_DIR):
    if subject.endswith(".json"):
        continue
    
    log_message = f"=====================================\n {subject}"
    print(log_message)
    log_output.append(log_message)
    
    # 3세트를 위한 리스트 초기화
    mock_exam_data = [[], [], []]
    exam_questions = 250
    for json_file in os.listdir(os.path.join(BASE_DIR, subject)):
        if not json_file.endswith("_mock_data.json"):
            continue
        
        with open(os.path.join(BASE_DIR, subject, json_file), 'r', encoding='utf-8') as f:
            json_file = json.load(f)
        
        domain = os.path.basename(f.name).replace("_mock_data.json", "")
        if (domain == '디지털') or (domain == '통계'):
            continue
        log_message = f"-------------------------------------\n {domain}"
        print(log_message)
        log_output.append(log_message)
    
        # subdomain 별로 모의고사 만들기 - 3세트용
        for subdomain in stats[subject][domain]['exam_subdomain_distribution'].keys():
            subdomain_data = [d for d in json_file if d['qna_subdomain'] == subdomain]
            random.shuffle(subdomain_data)
            
            needed_count = stats[subject][domain]['exam_subdomain_distribution'][subdomain]
            
            try:
                # 1세트 샘플링
                sample1 = random.sample(subdomain_data, needed_count)
                remaining_data = [d for d in subdomain_data if d not in sample1]
                
                # 2세트 샘플링 (1세트 제외한 데이터에서)
                sample2 = random.sample(remaining_data, needed_count)
                remaining_data = [d for d in remaining_data if d not in sample2]
                
                # 3세트 샘플링 (1, 2세트 제외한 데이터에서)
                sample3 = random.sample(remaining_data, needed_count)
                
                log_message = f" - {subdomain}: {needed_count} x 3세트 (총 {len(subdomain_data)}개 중 {needed_count * 3}개 사용)"
                print(log_message)
                log_output.append(log_message)
                
                # 각 세트에 추가
                mock_exam_data[0].extend(sample1)
                mock_exam_data[1].extend(sample2)
                mock_exam_data[2].extend(sample3)
                
            except ValueError:
                # 데이터가 부족한 경우
                total_available = len(subdomain_data)
                # per_set = total_available // 3
                sample1 = subdomain_data[:needed_count]
                sample2 = subdomain_data[needed_count:needed_count*2]
                sample3 = subdomain_data[needed_count*2:]
                
                log_message = f" - (ERROR) {subdomain}: {total_available}/{needed_count*3} (데이터 부족: {needed_count*3 - total_available}개 필요)"
                print(log_message)
                log_output.append(log_message)
                
                mock_exam_data[0].extend(sample1)
                mock_exam_data[1].extend(sample2)
                mock_exam_data[2].extend(sample3)
    
    # 3개 파일로 저장
    for set_num in range(3):
        log_message = f" ====> {set_num+1}세트: {len(mock_exam_data[set_num])}/{exam_questions} ({len(mock_exam_data[set_num])/exam_questions:.2%})"
        print(log_message)
        log_output.append(log_message)
        # 출력 디렉토리 생성
        os.makedirs(os.path.join(OUTPUT_DIR, mock_exam_name[set_num+1]), exist_ok=True)
        with open(f'{OUTPUT_DIR}/{mock_exam_name[set_num+1]}/{subject}_mock_exam.json', 'w', encoding='utf-8') as f:
            json.dump(mock_exam_data[set_num], f, ensure_ascii=False, indent=4)

# 로그 파일 저장
os.makedirs('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs', exist_ok=True)
with open(log_file, 'a', encoding='utf-8') as f:
    f.write('\n'.join(log_output))

print(f"\n로그가 저장되었습니다: {log_file}")

### multiple options 오류들

In [ ]:
import json
import re

# with open(os.path.join(BBASE_DIR, 'exam_statistics.json'), 'r', encoding='utf-8') as f:
    # stats = json.load(f)

# for subject in stats.keys():
    # print("================\n", subject)
    # with open(f'evaluation/eval_data/mock_exam/1st/{subject}_mock_exam.json', 'r', encoding='utf-8') as f:
        # data = json.load(f)

# file_name = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/1_filter/multiple.json'
file_name = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json'

with open(file_name, 'r', encoding='utf-8') as f:
    data = json.load(f)

if True:
    # ①②③④⑤로 시작하지 않는 경우를 찾기
    invalid_options = []

    for d in data:
        if d['options'] and isinstance(d['options'], list):
            options = d['options']
            
            # 각 옵션을 확인
            for i, option in enumerate(options):
                # ①②③④⑤로 시작하지 않는 경우 찾기
                if not re.match(r'^[①②③④⑤]', option.strip()):
                    invalid_options.append({
                        'file_id': d['file_id'],
                        'domain': d['domain'],
                        'subdomain': d['subdomain'],
                        'tag': d['tag'],
                        'option_index': i,
                        'original_option': option,
                        # 'first_char': option.strip()[0] if option.strip() else ''
                    })

    print(f"①②③④⑤로 시작하지 않는 options 수: {len(invalid_options)}")
    invalid_options

In [ ]:
invalid_options

## short/essay인 객관식 조정하기

In [ ]:
import os, json

with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/essay_subdomain_classified_ALL.json', 'r', encoding='utf-8') as f:
    essay = json.load(f)

with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/short_subdomain_classified_ALL.json', 'r', encoding='utf-8') as f:
    short = json.load(f)



In [ ]:
to_multiple = []

# for e in essay:
#     if isinstance(e['options'], list):
#         to_multiple.append(e)
# print(len(to_multiple))

new_short = []
for s in short:
    if isinstance(s['options'], list):
        to_multiple.append(s)
    else:
        new_short.append(s)
print(len(to_multiple), len(short), len(new_short))
# to_multiple

In [ ]:
with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/short_subdomain_classified_ALL.json', 'w', encoding='utf-8') as f:
    json.dump(new_short, f, ensure_ascii=False, indent=4)

with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_shortans_classified_ALL.json', 'w', encoding='utf-8') as f:
    json.dump(to_multiple, f, ensure_ascii=False, indent=4)

### 원본에 옮기기

In [ ]:
import os, time
from tqdm import tqdm

file_list = []

for t in tqdm(to_multiple):
    file_id = t.get('file_id')
    file_list.append(file_id)
    # print(file_id)
    tag = t.get('tag')
    time.sleep(0.5)
        
    file_path = os.popen(f"find /Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL -type f -name '{file_id}_v2.json'").read().strip()
    if file_path == "":
        file_path = os.popen(f"find /Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL -type f -name '{file_id}.json'").read().strip()
    # print(file_path)

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    contents = data['contents']

    for d in contents:
        if d.get('page') == tag.split('_')[1]:
            add_info = d.get('add_info')
            for info in add_info:
                if info.get('tag') == tag:
                    if info.get('description').get('question') == t.get('question'):
                        pass
                    else:
                        # print("질문 다름")
                        info['description']['question'] = t.get('question')

                    if info.get('description').get('answer') == t.get('answer'):
                        pass
                    else:
                        # print("답 다름")
                        info['description']['answer'] = t.get('answer')
                    if info.get('description').get('options') == t.get('options'):
                        pass
                    else:
                        # print("옵션 다름")
                        info['description']['options'] = t.get('options')
    if file_path.endswith("_v2.json"):
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
    else:
        with open(file_path.replace(".json", "_v2.json"), 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
# sorted(list(set(file_list)))

## multiple 실패한거

In [ ]:
# import os, json

# with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json', 'r', encoding='utf-8') as f:
#     multiple = json.load(f)

# # with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple-fail_response.json', 'r', encoding='utf-8') as f:
# #     multiple_fail = json.load(f)

In [ ]:
# for q in multiple:
#     # print(q)
#     if q['domain'] == '분류실패':
#         for m in multiple_fail:
#             if (m['file_id'] == q['file_id']) and (m['tag'] == q['tag']):
#                 q['domain'] = m['domain']
#                 q['subdomain'] = m['subdomain']
#                 q['classification_reason'] = m['classification_reason']
#                 q['is_calculation'] = m['is_calculation']
#                 # break
#     # break

In [ ]:
# with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json', 'w', encoding='utf-8') as f:
#     json.dump(multiple, f, ensure_ascii=False, indent=4)

In [ ]:
# import json
# with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple-fail_response_fail_again.json', 'r', encoding='utf-8') as f:
#     multiple_fail_again = json.load(f)

In [ ]:
# for q in multiple:
#     if q['domain'] == '':
#         for m in multiple_fail_again:
#             file_id = m['qna_id'].split('_')[0]
#             tag = m['qna_id'].replace(file_id+'_', '')
#             if (file_id == q['file_id']) and (tag == q['tag']):
#                 q['domain'] = m['domain']
#                 q['subdomain'] = m['subdomain']
#                 q['classification_reason'] = m['reason']
#                 q['is_calculation'] = m['is_calculation']
#                 # print(q)
#                 # break


## 통계

In [ ]:
import os, json
from tools.evaluation import qna_subdomain_classifier


with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json', 'r', encoding='utf-8') as f:
    multiple = json.load(f)

classifier = qna_subdomain_classifier.QnASubdomainClassifier()
classifier.save_statistics(multiple)

In [ ]:
for m in multiple:
    if m['subdomain'].count('-') >= 1:
        # print(m)
        # break
        m['subdomain'] = m['subdomain'].split('-')[0].strip()
        # print(m)
        # break
    elif m['subdomain'].count('.') >= 1:
        m['subdomain'] = m['subdomain'].split('.')[1].strip()
        

classifier.save_statistics(multiple)
    # print("없는데?", m['subdomain'])

with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json', 'w', encoding='utf-8') as f:
    json.dump(multiple, f, ensure_ascii=False, indent=4)


In [ ]:
with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json', 'r', encoding='utf-8') as f:
    multiple = json.load(f)

for m in multiple:
    m['domain'] = ""
    m['subdomain'] = ""
    m['classification_reason'] = ""
    m['is_calculation'] = False

with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple.json', 'w', encoding='utf-8') as f:
    json.dump(multiple, f, ensure_ascii=False, indent=4)


# 객관식 문제 변형

In [ ]:
import os, json

with open('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple.json', 'r', encoding='utf-8') as f:
    multiple = json.load(f)

pick_right = []
pick_wrong = []
pick_abcd = []

len(multiple)

In [ ]:
import re

# 리스트를 순회하면서 remove()를 하면 인덱스가 꼬이므로, 새로 분류하는 방식으로 변경
remaining = []
for m in multiple:
    question = m['question']
    
    # 3. ㄱ/ㄴ/ㄷ/ㄹ 등에서 옳은 것을 모두 고르는 문제 (다중 선택)
    # 종결어미 '다.' 제외: 줄바꿈, 문장 시작, 또는 공백 후에 나오고 뒤에 공백이 아닌 문자가 오는 경우만 매칭
    # 가나다라마바사아자차카타파하까지 포함 (바, 사, 아 등도 포함)
    if ('옳은 것을 모두 고른 것은?' in question) or \
         ('옳은 것을 모두 고르면?' in question) or \
         ('옳은 것을 모두 고른 것은' in question) or \
         ('옳은 것을 모두' in question and '?' in question) or \
         ('모두 고른 것은?' in question) or \
         ('모두 고르면?' in question) or \
         ('모두 고른 것은' in question) or \
         ('모두 묶인 것은?' in question) or \
         (re.search(r'[ㄱㄴㄷㄹ][\.]', question) and ('모두 고른' in question or '모두 묶인' in question)) or \
         (re.search(r'(?:^|\n| )[가나다라마바사아자차카타파하]\.(?!\s|$)', question) and ('모두 고른' in question or '모두 묶인' in question)) or \
         (re.search(r'[㉠㉡㉢㉣㉤]', question) and ('모두 고른' in question or '모두 묶인' in question)) or \
         (re.search(r'[ⓐⓑⓒⓓⓔ]', question) and ('모두 고른' in question or '모두 묶인' in question)):
        pick_abcd.append(m)
    # 1. 옳은 것을 고르는 문제 (단일 선택)
    # if ('옳은' in question) or ('옳게' in question) or ('해당하는' in question) or ('적절한' in question) or ('적절하게' in question) or ('바르게' in question) or ('올바른' in question) or ('가장 깊은' in question) or ('가장 타당한' in question) or ('관련 있는' in question):
    #     pick_right.append(m)
    # 2. 옳지 않은 것을 고르는 문제 (단일 선택)
    elif ('않은' in question) or ('못한' in question) or ('없는' in question) or ('거리가 먼' in question) or ('아닌' in question) or ('아니하는 것' in question) or ('않는' in question) or ("않게" in question) or ('잘못된' in question) or ('틀린' in question) or ('다른' in question) or ('무관한' in question) or ('가장 먼' in question) or ('어려운' in question):
        pick_wrong.append(m)
    else:
        # 분류되지 않은 항목
        # remaining.append(m)
        pick_right.append(m)

# 분류 결과 확인
print(f"전체: {len(multiple)}")
print(f"옳은 것은? (단일 선택): {len(pick_right)}")
print(f"옳지 않은 것은? (단일 선택): {len(pick_wrong)}")
print(f"옳은 것을 모두 고른 것은? (다중 선택): {len(pick_abcd)}")
print(f"미분류: {len(remaining)}")
print(f"합계: {len(pick_right) + len(pick_wrong) + len(pick_abcd) + len(remaining)}")

len(remaining), len(pick_right), len(pick_wrong), len(pick_abcd)

In [ ]:
# for r in remaining:
#     print(r['question'])
#     print("---------")

In [ ]:
pick_abcd

## 옳지 않은 것

In [ ]:
pick_wrong